# Yes24 크롤링

## 개요

1. 구글 홈페이지의 임의의 class attribute로 html 컨텐츠 가져오기
2. Yes24 베스트셀러 컨텐츠 가져오기
3. 최대 3페이지까지 가지고 오기


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import re
from datetime import datetime

## webdriver manager 불러오기
ChromeDriverManager().install()

## 자동화 크롬 실행하기
browser = webdriver.Chrome()
browser.get("https://google.com")

여기까지 하면 아래와 같이 새 크롬 탭이 열리는 것을 볼 수 있다:

![resource/Screenshot 2024-04-08 at 21.47.50.png](./resource/Screenshot%202024-04-08%20at%2021.47.50.png)

그럼 이제 구글 페이지에서 아무 컴포넌트의 텍스트를 불러와보자. 하단에 있는 "대한민국" 버튼을 불러보자.


In [18]:
## google 페이지에서 "대한민국" 가져오기

class_name = "uU7dJb"

element = browser.find_element(By.CLASS_NAME, class_name)

print(element.text)

대한민국


## 크롤링을 사용하여 Yes24 베스트 셀러 데이터 수집하기

1. yes24 베스트셀러 항목 보기로 간다.
2. 모든 베스트셀러 항목들의 상세 페이지를 순회한다.
3. 3페이지까지 반복한다.

### 수집할 정보들

- 책 제목 (title)
- 저자 (author)
- 출판사 (publisher)
- 출판일 (publishing)
- 평점 (rating)
- 리뷰 (review)
- 판매지수 (sales)
- 가격 (price)
- 국내도서랭킹 (ranking)
- 국내도서TOP100 (ranking_weeks)


In [19]:
link_list = []

## 1 페이지의 링크 데이터 수집

# elements = browser.find_elements(By.CLASS_NAME, class_name)

# for element in elements:
#     print(element.get_attribute("href"))

## 1 ~ 3 페이지의 링크 데이터 수집
for i in range(1, 4):
    url = f"https://www.yes24.com/Product/Category/BestSeller?pageNumber={i}&pageSize=24"

    browser.get(url)

    elements = browser.find_elements(By.CLASS_NAME, "gd_name")
    for element in elements:
        link_list.append(element.get_attribute("href"))
        print(element.text, end=": \n")
        print(element.get_attribute("href"), "\n")

    sleep(0.2)

나는 읽고 쓰고 버린다: 
https://www.yes24.com/Product/Goods/126010792 

불변의 법칙: 
https://www.yes24.com/Product/Goods/124999476 

흔한남매 16: 
https://www.yes24.com/Product/Goods/126029503 

나를 소모하지 않는 현명한 태도에 관하여: 
https://www.yes24.com/Product/Goods/125295101 

빨모쌤의 라이브 영어회화: 
https://www.yes24.com/Product/Goods/125992096 

일류의 조건: 
https://www.yes24.com/Product/Goods/125491624 

삶이 흔들릴 때 뇌과학을 읽습니다: 
https://www.yes24.com/Product/Goods/125651056 

4~7세 보고 만지는 수학은 이렇게 가르칩니다: 
https://www.yes24.com/Product/Goods/126110769 

세이노의 가르침: 
https://www.yes24.com/Product/Goods/117014613 

꽃길이 따로 있나, 내 삶이 꽃인 것을: 
https://www.yes24.com/Product/Goods/125576919 

나는 메트로폴리탄 미술관의 경비원입니다: 
https://www.yes24.com/Product/Goods/123675187 

마흔에 읽는 쇼펜하우어: 
https://www.yes24.com/Product/Goods/122120495 

던전밥 14: 
https://www.yes24.com/Product/Goods/126029190 

마이크로 리추얼 : 사소한 것들의 힘: 
https://www.yes24.com/Product/Goods/125973831 

삼체 1~3 세트: 
https://www.yes24.com/Product/Goods/90926731 

이처럼 사소한 것들: 
https://www.yes24

## Chapter 5-32. 예스24 베스트셀러 데이터 수집 후 데이터 분석하기


In [20]:
link_list

['https://www.yes24.com/Product/Goods/126010792',
 'https://www.yes24.com/Product/Goods/124999476',
 'https://www.yes24.com/Product/Goods/126029503',
 'https://www.yes24.com/Product/Goods/125295101',
 'https://www.yes24.com/Product/Goods/125992096',
 'https://www.yes24.com/Product/Goods/125491624',
 'https://www.yes24.com/Product/Goods/125651056',
 'https://www.yes24.com/Product/Goods/126110769',
 'https://www.yes24.com/Product/Goods/117014613',
 'https://www.yes24.com/Product/Goods/125576919',
 'https://www.yes24.com/Product/Goods/123675187',
 'https://www.yes24.com/Product/Goods/122120495',
 'https://www.yes24.com/Product/Goods/126029190',
 'https://www.yes24.com/Product/Goods/125973831',
 'https://www.yes24.com/Product/Goods/90926731',
 'https://www.yes24.com/Product/Goods/123400303',
 'https://www.yes24.com/Product/Goods/96547408',
 'https://www.yes24.com/Product/Goods/126012006',
 'https://www.yes24.com/Product/Goods/125557465',
 'https://www.yes24.com/Product/Goods/123930880',
 '

In [24]:
find_by_class = lambda class_name: browser.find_element(By.CLASS_NAME, class_name).text


def extract_number(s: str) -> int:
    numbers = re.findall(r"\d+", s)
    if numbers:
        numbers = "".join(numbers)
        return int(numbers)
    return 0


books = []
for link in link_list:
    browser.get(link)

    ranks = (
        browser.find_element(By.CLASS_NAME, "gd_best")
        .find_element(By.TAG_NAME, "dd")
        .text.split(" | ")
    )
    publishing = find_by_class("gd_date")
    review = extract_number(find_by_class("gd_reviewCount"))
    sales = extract_number(find_by_class("gd_sellNum"))

    # rating
    try:
        rating = (
            browser.find_element(By.ID, "spanGdRating").find_element(By.CLASS_NAME, "yes_b").text
        )
        rating = float(rating)
    except:
        rating = None

    # publishing
    match = re.match(r"(\d+)년 (\d+)월 (\d+)일", publishing)
    if match:
        year, month, day = (int(x) for x in match.groups())
        date_obj = datetime(year, month, day)
        publishing = date_obj.strftime("%Y-%m-%d")
    else:
        publishing = None

    book = {
        "title": find_by_class("gd_name"),
        "auth": find_by_class("gd_auth"),
        "publisher": find_by_class("gd_pub"),
        "publishing": publishing,
        "rating": rating,
        "review": review,
        "sales": sales,
        "price": int(browser.find_element(By.CLASS_NAME, "yes_m").text[:-1].replace(",", "")),
        "ranking": int(ranks[0][-2:-1]),
        "ranking_weeks": int(ranks[1][-2:-1]) if len(ranks) > 1 else None,
    }

    print(book)
    books.append(book)

    sleep(0.1)

{'title': '나는 읽고 쓰고 버린다', 'auth': '손웅정 저', 'publisher': '난다', 'publishing': '2024-04-20', 'rating': 9.9, 'review': 17, 'sales': 166470, 'price': 17000, 'ranking': 1, 'ranking_weeks': 1}
{'title': '불변의 법칙', 'auth': '모건 하우절 저/이수경 역', 'publisher': '서삼독', 'publishing': '2024-02-28', 'rating': 9.7, 'review': 146, 'sales': 502701, 'price': 25000, 'ranking': 2, 'ranking_weeks': 6}
{'title': '흔한남매 16', 'auth': '흔한남매 원저/백난도 글/유난희 그림/흔한컴퍼니 감수', 'publisher': '미래엔아이세움', 'publishing': '2024-04-25', 'rating': 10.0, 'review': 2, 'sales': 96600, 'price': 14500, 'ranking': 3, 'ranking_weeks': 1}
{'title': '나를 소모하지 않는 현명한 태도에 관하여', 'auth': '마티아스 뇔케 저/이미옥 역', 'publisher': '퍼스트펭귄', 'publishing': '2024-03-10', 'rating': 9.6, 'review': 64, 'sales': 315012, 'price': 17800, 'ranking': 4, 'ranking_weeks': 5}
{'title': '빨모쌤의 라이브 영어회화', 'auth': '신용하 저', 'publisher': '웅진지식하우스', 'publishing': '2024-04-24', 'rating': 9.7, 'review': 27, 'sales': 108714, 'price': 22000, 'ranking': 5, 'ranking_weeks': 1}
{'title': '일류

## pymysql과 연동하기


In [ ]:
import pymysql
from pymysql.cursors import DictCursor

with pymysql.connect(
    host="localhost",
    user="root",
    password="oz-password",
    db="yes24",
    charset="utf8mb4",
    cursorclass=DictCursor,
) as conn:
    with conn.cursor() as cur:
        for book in books:
            sql = """INSERT INTO Books(
                title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

            cur.execute(sql, tuple(book.values()))

        conn.commit()